In [1]:
import torch 
import numpy as np
import pandas as pd
import tensorboard
from torch.nn.utils.rnn import pad_sequence


In [2]:
import sys
sys.path.append('C:/Users/Owner/ICT5012 - Disseration/transit_learning-master_Newest')
from torch_geometric.loader import DataLoader
from simulation.citygraph_dataset import CityGraphData, \
    get_dataset_from_config, STOP_KEY
from simulation.transit_time_estimator import RouteGenBatchState, get_cost_module_from_cfg
import learning.utils as lrnu
from torch_utils_2 import get_batch_tensor_from_routes, dump_routes
from tqdm import tqdm
import time

In [3]:
# Solution Mater Dei North Routes
Mater_Dei_NorthRoutes = [
    torch.tensor([3,81,75,76,77,78,79,80,85,86,87,88,119,194,195,196,197,198,29,199,200,201,202,203,204,205,206,207,208]),
    torch.tensor([208,209,210,211,201,200,212,213,196,197,214,193,215,28,116,117,118,88,87,86,77,79,80,84,76,75,0,1,2]),
    torch.tensor([202,203,204,205,206,207,208,209,210,211,201,200,212,213,196,197,214,193,215,28,116,117,118,88,87,86,77,79,80,84,76,75,0,1,2]),
    torch.tensor([3,269,22,23,24,25,26,27,28,29,30,140,31,141,32,33,142,143,144,145,150,144,152,153,154,155,186,187,188,156,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184]),
    torch.tensor([184,216,217,218,185,219,220,221,222,223,186,187,188,155,154,153,144,51,150,144,143,142,52,32,141,53,151,30,29,28,54,26,25,24,23,22,5,0,1,2]),
    torch.tensor([174,175,176,177,178,179,180,182,183,184,216,217,218,185,219,220,221,222,223,186,187,188,155,154,153,144,51,150,144,143,142,270,32,141,53,151,30,29,28,54,26,25,24,23,22,5,0,1,2]),
    torch.tensor([3,269,139,138,137,136,123,124,125,126,127,128,129,130,131,132,133,134,27,28,29,30,140,31,141,32,33,142,143,144,145,146,34,147,148,157,189,190,191,192,148,158,159,160,161,162,163,164,165]),
    torch.tensor([165,166,167,168,161,160,158,147,148,157,189,190,191,192,148,34,149,51,150,144,143,142,52,32,141,53,151,30,29,28,54,134,133,132,135,130,129,128,126,125,123,136,137,138,139,5,0,1,2]),
    torch.tensor([158,159,160,161,162,163,164,165,166,167,168,161,160,158,147,148,157,189,190,191,192,148,34,149,51,150,144,143,142,52,32,141,53,151,30,29,28,54,134,133,132,135,130,129,128,126,125,123,136,137,138,139,5,0,1,2]),
    torch.tensor([3,269,139,138,137,136,123,224,225,226,227,228,229,230,231,232,233,234,235,236,237,239,240,241,240,239,238,248,249,250,251,252,253,254,255]),
    torch.tensor([255,256,257,258,259,260,261,262,263,264,265,249,266,267,239,240,241,240,239,238,237,236,235,242,232,243,244,245,229,228,268,246,225,247,224,123,136,137,138,139,5,0,1,2]),
    torch.tensor([3,81,82,56,62,58,21,20,19,57,61,14,13,12,11,10,9,8,7,6,63,64,65,66,67,68,69,70,71]),
    torch.tensor([71,72,71,73,74,4,55,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,58,59,60,83,0,1,2]),
    torch.tensor([63,64,65,66,67,68,69,70,71,72,71,73,74,4,55,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,58,59,60,83,0,1,2]),
    torch.tensor([3,269,83,56,110,111,112,113,114,115,27,116,117,118,120,121,89,90,91,92,93,94,95,96,97,35,98,99,100,101,102,36,37,38,39,40,41,42,43]),
    torch.tensor([103,44,45,46,47,48,49,50,102,101,100,99,98,35,97,104,95,105,106,107,108,109,271,121,120,118,117,116,122,54,115,114,113,112,111,110,83,0,1,2])
]

In [4]:
class Config(dict):
    def __init__(self, *args, **kwargs):
        super(Config, self).__init__(*args, **kwargs)
        for key, value in self.items():
            if isinstance(value, dict):
                self[key] = Config(value)
            elif isinstance(value, list):
                # Convert dict elements in lists to Config objects
                self[key] = [Config(item) if isinstance(item, dict) else item for item in value]

    __getattr__ = dict.get

cfg = Config({
    'ppo': {
        'n_iterations': 200,
        'val_period': 10,
        'n_epochs': 1,
        'minibatch_size': 2,
        'horizon': 26,
        'epsilon': 0.2,
        'use_gae': False,
        'gae_lambda': 0.95
    },
    'discount_rate': 0.95,
    'diff_reward': False,
    'baseline_lr': 0.0005,
    'entropy_weight': 0,
    'batch_size': 2,
    'reward_scale': 1.0,
    'lr': 0.0016134816080499328,
    'decay': 0.0008404361781997002,
    'optimizer': 'Adam',
    'eval': {
        'n_routes': 15,
        'min_route_len': 11,
        'max_route_len': 42
    },
    'dataset': {
        'type': 'mumford',
        'kwargs': {
            'path': 'datasets/mumford_dataset/Instances',
            'city': 'Gozo'
        }
    },
    'experiment': {
        'logdir': 'training_logs',
        'anomaly': False,
        'cpu': False,
        'seed': 0,
        'symmetric_routes': False
    },
    'defaults': [
        '_self_',
        {
            'cost_function': {
                'type': 'mine',
                'kwargs': {
                    'mean_stop_time_s': 0,
                    'avg_transfer_wait_time_s': 300,
                    'demand_time_weight': 0.5,
                    'route_time_weight': 0.5,
                    'constraint_violation_weight': 5.0,
                    'variable_weights': False,
                    'pp_fraction': 0.33,
                    'op_fraction': 0.33
                }
            }
        }
    ]
})


In [5]:
class Config(dict):
    def __init__(self, *args, **kwargs):
        super(Config, self).__init__(*args, **kwargs)
        for key, value in self.items():
            if isinstance(value, dict):
                self[key] = Config(value)
    __getattr__ = dict.get

# Minimal experiment configuration
exp_dc = Config({
    'cost_function': {
        'type': 'mine',  # Specify the cost function type as 'mine'
        'kwargs': {}     # Any additional parameters can be added here
    },
    'symmetric_routes': False,    # or False, based on your requirement
    'low_memory_mode': False
})


In [6]:
class Config(dict):
    __getattr__ = dict.get

# ANDRE - CHANGE HERE
Dataset_Info = Config({
    'csv': True,
    'n_routes': 16,
    'min_route_len': 29,
    'max_route_len': 56,
    'type': 'mumford',
    'path': 'C:/Users/Owner/ICT5012 - Disseration/transit_learning-master/CEC2013Supp/Instances/Evening_Model',
    'city': 'Mater_dei_north_routes'
})

In [7]:
exp_cfg = cfg.experiment

In [8]:
if exp_cfg.get('cpu', False) or not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device("cuda")

In [9]:
if 'model' in cfg:
    # setup the model
    model = build_model_from_cfg(cfg['model'], exp_cfg)
    if 'weights' in cfg.model:
        model.load_state_dict(torch.load(cfg.model.weights,map_location=device))
    elif weights_required and cfg.model.route_generator.type != 'RandomPathCombiningRouteGenerator': raise ValueError("model weights are required but not provided")
else:
    model = None

In [10]:
# setup the cost function
low_mem_mode = exp_cfg.get('low_memory_mode', False)
cost_obj = get_cost_module_from_cfg(exp_dc.cost_function, low_mem_mode,
                                    exp_cfg.symmetric_routes)

low_memory_mode: False


In [11]:
cost_obj.to(device)
if model is not None:
    model.to(device)

In [12]:
test_ds = get_dataset_from_config(Dataset_Info)
test_dl = DataLoader(test_ds, batch_size=10)


In [13]:
n_samples = cfg.get('n_samples', None)

In [14]:
sbs = cfg.get('sample_batch_size', cfg.batch_size)

In [15]:
for data in tqdm(test_dl):
    if device is not None and device.type != 'cpu':
        data = data.cuda()
    start_time = time.time()
    state = RouteGenBatchState(data, cost_obj, cfg.eval.n_routes, 
                                   cfg.eval.min_route_len, 
                                   cfg.eval.max_route_len)

100%|██████████| 1/1 [00:02<00:00,  2.91s/it]


In [16]:
import torch

# Your list of route tensors (on CUDA). Each tensor is 1D.
routes_list = Mater_Dei_NorthRoutes

# Determine maximum route length among all routes.
max_length = max(route.numel() for route in routes_list)
print("Max route length:", max_length)

# Pad each route with -1 so that all routes have the same length.
padded_routes = []
for route in routes_list:
    pad_length = max_length - route.numel()
    if pad_length > 0:
        padded_route = torch.cat([route, -1 * torch.ones(pad_length, dtype=route.dtype, device=route.device)])
    else:
        padded_route = route
    padded_routes.append(padded_route)

# Stack the padded routes to form a tensor of shape [n_routes, max_length].
routes_tensor = torch.stack(padded_routes)

# Add a batch dimension so the final tensor has shape [1, n_routes, max_length].
final_routes_tensor_correct = routes_tensor.unsqueeze(0)


Max route length: 56


In [17]:
state.add_new_routes(final_routes_tensor_correct)


In [18]:
MEAN_STOP_TIME_S = 0
AVG_TRANSFER_WAIT_TIME_S = 300
UNSAT_PENALTY_EXTRA_S = 3000
EPSILON = 1e-6


# Start by Obtaining Passenger Cost

# Load Demand Matrix from State
demand_matrix = state.demand
# Load Drive Times Matrix from State
drive_times_matrix = state.drive_times

# Calculate amount of demand at each number of transfers
trips_at_transfers = torch.zeros(state.batch_size, 4, device=device)
# Load Number of transfers
n_transfers = state.n_transfers.clone()
# Label instances where no path exists as nopath
nopath = ~state.has_path
# Obtain number of unsatisfied demand
n_transfers[nopath] = 3
for ii in range(3):
    d_i = (demand_matrix * (n_transfers == ii)).sum(dim=(1,2))
    trips_at_transfers[:, ii] = d_i
    
d_un = (demand_matrix * (n_transfers > 2)).sum(dim=(1, 2))
trips_at_transfers[:, 3] = d_un


# Load transit_times (equivalent to shortest distances based on floyd_warshall)
trip_times = state.transit_times
# Adjust 'inf' trip_time values to 0 if nopath exists
trip_times[nopath] = 0
# Caclulate demand_time matrix
demand_time = demand_matrix * trip_times
# Calculate total_demand_time
total_dmd_time = demand_time.sum(dim=(1, 2))
# Calculate number of total_transfers
demand_transfers = demand_matrix * state.n_transfers
total_transfers = demand_transfers.sum(dim=(1, 2))
# Calculate unserved demand
unserved_demand = (demand_matrix * nopath).sum(dim=(1, 2))
# Calculate total demand
total_demand = demand_matrix.sum(dim=(1,2))
# Calculate served demand
served_demand = (state.has_path * demand_matrix).sum(dim = (1,2))
# Calculate Mean Demand Time
mean_demand_time = total_dmd_time/ (served_demand + EPSILON)
# Calculate time_normalizer
time_normalizer = state.drive_times.flatten(1,2).max(1).values
# Calculate Demand Cost
demand_cost = (mean_demand_time*served_demand) + (unserved_demand*time_normalizer*2)
# Calculate Demand Cost (Before Normalizer)
demand_cost /= total_demand
# Calculate Demand Cost (After Normalizer)
demand_cost = demand_cost/time_normalizer

# Start by Obtaining Operator Cost
n_routes = state.n_routes_to_plan
# Calculate Route Cost (Before Normalizer)
route_cost = state.total_route_time
# Calculate Route Cost (After Normalizer)
route_cost = route_cost/(time_normalizer * n_routes + 1e-6)

# Passenger and Operator Cost
demand_weight = 0.5
route_weight = 0.5
Passenger_y_Operator_Cost = (demand_weight*demand_cost) + (route_weight*route_cost)


# Calculate Constraint Violation

# Number of Uncovered Demand Nodes
frac_uncovered = state.get_n_disconnected_demand_edges() / state.n_demand_edges
# Calculate Constraint Violation
const_viol_cost = frac_uncovered + 0.1 * (frac_uncovered > 0)

# Calculate final cost
cost = Passenger_y_Operator_Cost + (const_viol_cost * 5)

In [19]:
cost = cost
C_p = mean_demand_time/60
C_o = state.total_route_time/60
d_0 = (trips_at_transfers[:, 0]/total_demand)*100
d_1 = (trips_at_transfers[:, 1]/total_demand)*100
d_2 = (trips_at_transfers[:, 2]/total_demand)*100
d_unsat = (d_un/total_demand)*100
disconnected_node_pairs = state.get_n_disconnected_demand_edges()
stops_out_of_bounds = min(state.max_route_len - max_length,0)

In [20]:
# Create a dictionary with your desired column headers.
data = {
    "Cost": cost[0].item(),
    "C_p": C_p[0].item(),
    "C_o": C_o[0].item(),
    "0 Tr. (%)": d_0[0].item(),
    "1 Tr. (%)": d_1[0].item(),
    "2 Tr. (%)": d_2[0].item(),
    "Unsat. Demand (%)": d_unsat[0].item(),
    "Disc. NP": disconnected_node_pairs[0].item(),
    "Max RL": stops_out_of_bounds != 0
}

# Create a pandas DataFrame.
df = pd.DataFrame([data])  # each key becomes a column header

# Display the DataFrame.
print(df)

       Cost        C_p          C_o  0 Tr. (%)  1 Tr. (%)  2 Tr. (%)  \
0  2.259468  20.515188  1043.883301  36.739292   19.30879  13.598798   

   Unsat. Demand (%)  Disc. NP          Max RL  
0          30.353117       309  [tensor(True)]  


In [21]:
from tabulate import tabulate 
print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

+---------+---------+---------+-------------+-------------+-------------+---------------------+------------+----------+
|    Cost |     C_p |     C_o |   0 Tr. (%) |   1 Tr. (%) |   2 Tr. (%) |   Unsat. Demand (%) |   Disc. NP |   Max RL |
|---------+---------+---------+-------------+-------------+-------------+---------------------+------------+----------|
| 2.25947 | 20.5152 | 1043.88 |     36.7393 |     19.3088 |     13.5988 |             30.3531 |        309 |        1 |
+---------+---------+---------+-------------+-------------+-------------+---------------------+------------+----------+
